In [1]:
import numpy as np
import gudhi as gd

In [2]:
import mat73
def data():
    Data=mat73.loadmat('/scratch/mb2864/Data/GE_Data_221013_normPH.mat', use_attrdict=True)
    return Data

In [3]:
Data = data()

In [19]:
#Persistence Diagrams in a list
ListOfPDs=[]
for j in range(850):
    pers_diag=Data.X.h0_t20_n_flip[j]
    pers_diag=np.array(pers_diag)
    ListOfPDs.append(pers_diag)

In [27]:

from gudhi.representations import PersistenceImage

#Persistence Images

PI = PersistenceImage(bandwidth=0.5, weight=lambda x: 1/(1+np.exp(-x[1]+x[0])), \
                                       im_range=[-4,3,-3,3],   resolution=[50,50])
pi=PI.fit_transform(ListOfPDs)

In [9]:
def coord_matrix():
    
    #gudhi.plot_persistence_diagram(data.X.h0_t20[2])
    
    f1=np.zeros([850,1])
    f2=np.zeros([850,1])
    f3=np.zeros([850,1])
    f4=np.zeros([850,1])
    f5=np.zeros([850,1])
    f6=np.zeros([850,1])
    M= np.zeros([850,10])
    
    for k in range(2):
        for j in range(850):
            if k==0:
                H0_pers=Data.X.h0_t20_n[j]
            if k==1:
                H0_pers=Data.X.h1_t20_n[j]
            H0_pers=np.array(H0_pers)    
            p=[H0_pers[i,1]-H0_pers[i,0] for i in range(len(H0_pers))]
            
            p=np.array(p)
            
            b=H0_pers[:,0]
            d=H0_pers[:,1]
            b=np.array(b)
            d=np.array(d)
            d_max=np.max(d)
            #Carlson Coordinates
            f1[j]=np.sum(d*p)/len(d) 
            f2[j]=(d_max*np.sum(p)-np.sum(d*p))/len(d)
            f3[j]=np.sum(b**2*(p)**4)/len(d)
            f4[j]=d_max**2*np.sum((1-d/d_max)**2*p**4)/len(d)
            f5[j]=np.max(p)
            #f6[j]=np.sum(1/p)
        if k==0:
            M[:,0:5]=np.concatenate((f1,f2,f3,f4,f5),axis=1)
        else:
            M[:,5:10]=np.concatenate((f1,f2,f3,f4,f5),axis=1)

    return(M)

In [12]:
SurfaceStacked=np.zeros([40000,850])
for i in range(850):
    Surface=np.array(Data.X.hknorm[i])
    Surface=Surface[:,:,19]
    SurfaceColumn=np.reshape(Surface.T,(40000,1))
    SurfaceStacked[:,i] = SurfaceColumn.T

In [28]:
from sklearn.preprocessing   import MinMaxScaler
from sklearn.pipeline        import Pipeline
from sklearn.svm             import SVC
from sklearn.ensemble        import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from gudhi.representations import PersistenceImage


#X=ListOfPDs
X=pi
#X=coord_matrix()
#X=SurfaceStacked.T

lbls=np.zeros([850,1])
for j in range(17):
    lbls[50*j:50*j+50]=j*np.ones([50,1])

X_train, X_test, y_train, y_test = train_test_split(
    X, lbls, test_size=0.2, random_state=0)

classifiers = [
    Pipeline(
        [
            #("preprocess", PersistenceImage()),
            ("knn", KNeighborsClassifier())
        ]
    ),
    Pipeline(
        [   #("preprocess", PersistenceImage()),
            ("svm", SVC())
        ]
    ),
    Pipeline(
        [   #("preprocess", PersistenceImage()),
            ("rfc", RandomForestClassifier())
        ]
    ),
    Pipeline(
        [   #("preprocess", PersistenceImage()),
            ("nnet", MLPClassifier())
        ]
    ),
]

names=["KNN","SVC","RandomForest","Neural Net"]

param_grid=[#{'preprocess__bandwidth':[1], 'preprocess__weight':[lambda x: np.tanh(x[1])],'preprocess__im_range':\
             #[[-4,3,-3,3]],   'preprocess__resolution':[[50,50]]},
            {'knn__n_neighbors': [1, 3, 5, 7, 9, 11], 'knn__algorithm': ['auto']},
            {'svm__C': [1, 10], 'svm__kernel': ['linear', 'rbf'],'svm__gamma':['auto','scale']},
            {'rfc__criterion': ['gini','entropy']},
            {'nnet__alpha': [0.0001],'nnet__max_iter':[1000,2000]}]


for name,clf,params in zip(names,classifiers,param_grid):
    model = GridSearchCV(clf, params, cv=5)
    model_fit= model.fit(X_train,np.ravel(y_train))
    print(model.best_params_)
    score= model_fit.score(X_test, y_test)
    print("{} score: {}".format(name, score))
    

{'knn__algorithm': 'auto', 'knn__n_neighbors': 11}
KNN score: 0.711764705882353
{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
SVC score: 0.7235294117647059
{'rfc__criterion': 'gini'}
RandomForest score: 0.6882352941176471
{'nnet__alpha': 0.0001, 'nnet__max_iter': 1000}
Neural Net score: 0.38823529411764707


In [37]:
np.shape(ListOfPDs)

(850,)

In [ ]:
gd.representations.preprocessing.ProminentPoints